In [1]:
import pandas as pd 

In [2]:
def get_rows_year(annee,df):
    debut = f"{annee}-01-01"
    fin = f"{annee}-12-31"
    tmp_df = df.loc[(df['date'] > debut) & (df['date'] < fin)]
    return tmp_df

In [3]:
def mean_economy_match(df_economy):
    df_economy = df_economy.assign(economy_1="",economy_2="")
    match_id = df_economy["match_id"].drop_duplicates().tolist()
    for i in range(len(df_economy)):
        economy_1,economy_2, cpt=0,0,0
        
        for j in range(9,39):
            value = df_economy.iloc[i,j]
            cpt+=1
            if (value>=0):
                economy_1 += value
            else:
                break
                
        for j in range(40,70):
            value = df_economy.iloc[i,j]
            if (value>=0):
                economy_2 += value
            else:
                break
        df_economy.at[i,"economy_1"], df_economy.at[i,"economy_2"]  = (economy_1/cpt)/5,(economy_2/cpt)/5     
        
    return df_economy 

In [8]:
"""
    @Function_name : 
        - players_to_match
    @Parameters : 
        - [dataframe] df_players :
    @Return :
        - [dataframe] dataframe ['date','event_id','match_id','team_1','team_2','country_1','country_2','kda_1','kda_2','hs_1','hs_2']
    @Description : 
        Dans df_players on a pour un match les statistiques de chaque équipes.
        Soit 10 lignes dont 5 qui appartiennent à l'équipe 1 et les 5 autres à l'équipe 2.
        Le but de cette fonction est de calculer le kda moyen de chacune des équipes et de les regrouper sur une ligne pour un match.
        On stocke les résultats dans le nouveau dataframe qu'on a construit.
"""

def players_to_match(df_players):
    match_id = df_players["match_id"].drop_duplicates().tolist() #liste des ids des matchs présent dans la table players
    #on créé un nouveau dataframe qui contiendra toutes les informations relatives aux equipes pendant un match
    dataframe = pd.DataFrame(columns=['date','event_id','match_id','team_1','team_2','country_1','country_2','kda_1','kda_2','hs_1','hs_2'])
    m = 0 #initialisation des lignes de notre nouveaux dataframe pour savoir où on en est
    
    #pour tous les matchs dans players on effectue le traitement ci-dessous
    for idm in match_id: 
        df = df_players[df_players["match_id"] == idm] #permet d'obtenir les 10 lignes relatives au match d'id idm
        #on initialise les variables qu'on va calculer (kda = ratio kills/deaths/assists, hs = headshot, t1(resp 2) = name_team_1(2), l = lines in df 
        kda1, kda2, hs1, hs2, t1, t2, l = 0, 0, 0, 0, df.iloc[0,2], df.iloc[0,3],0 
        dataframe.at[m,"date"],dataframe.at[m,"event_id"],dataframe.at[m,"match_id"] = df.iloc[0,0], df.iloc[0,7], df.iloc[0,6] #on set déjà date, match id et event id
        
        #pour un match qui a bien 10 joueurs
        if len(df)==10:
            for i in range(0,10):
                #on trouve le nom de la team, ces kills, assists, deaths et head shot 
                t, k, a, d, hs = df.iloc[i,2], df.iloc[i,13], df.iloc[i,14], df.iloc[i,15], df.iloc[i,16]
                
                if t == t1:
                    #si dans team il s'agit de t1 alors on calcul son kda / hs qu'on ajoute à kda1 et hs1 ainsi que son pays
                    kda1 = kda1 + k + a if d == 0 else kda1 + (k + a)/d
                    hs1 = hs1 + hs
                    dataframe.at[m,"country_1"] = df.iloc[l,4]
 
                else: 
                    #inversement pour kda2,hs2 et le pays
                    kda2 = kda2 + k + a if d == 0 else kda2 + (k + a)/d
                    hs2 = hs2 + hs
                    dataframe.at[m,"country_2"] = df.iloc[l,4]
                l += 1 #on avance d'une ligne 
            #on oublie par d'ajouter à la ligne m de notre nouveaux dataframe les valeurs qu'on vient de calculer 
            dataframe.at[m,"team_1"],dataframe.at[m,"team_2"] = t1,t2
            dataframe.at[m,"kda_1"],dataframe.at[m,"kda_2"]  = kda1/5,kda2/5
            dataframe.at[m,"hs_1"], dataframe.at[m,"hs_2"] = hs1/5, hs2/5
            m += 1 #on change de match et donc de ligne dans notre dataframe 
    return dataframe  

In [5]:
def switch_teams_res_eco(df_res,df_eco):  
    match_id = df_res["match_id"].drop_duplicates().tolist()
    for m in match_id:
        df1, df2 = df_res[df_res["match_id"]==m], df_eco[df_eco["match_id"]==m]
        ldf1, ldf2 = df1.index.tolist(), df2.index.tolist()
        sdf1, sdf2 = len(ldf1), len(ldf2)
        if sdf1 != 0 and sdf2 != 0:
            for l1 in range(sdf1):
                for l2 in range(sdf2):
                    if (df1.iloc[l1,3] == df2.iloc[l2,5]) and (df1.iloc[l1,1] == df2.iloc[l2,4]):
                        df_eco.at[ldf2[l2],"team_1"] = df1.iloc[l1,2]
                        df_eco.at[ldf2[l2],"team_2"] = df1.iloc[l1,1]
                        tmp = df2.iloc[l2,6]
                        df_eco.at[ldf2[l2],"economy_1"] = df2.iloc[l2,7]
                        df_eco.at[ldf2[l2],"economy_2"] = tmp
    return df_eco

In [3]:
def ratio_result(df_res):  
    new = pd.DataFrame(columns=["date","_map","result_p1","economy","rank","kda","match_winner","hs"])
    for l in range(len(df_res)):
        new.at[l,"date"]= df_res.iloc[l,0]
        new.at[l,"_map"]= df_res.iloc[l,1]
        new.at[l,"match_winner"]= df_res.iloc[l,8]
        new.at[l,"result_p1"]=0 if df_res.iloc[l,3]==0 else df_res.iloc[l,2]/df_res.iloc[l,3]
        new.at[l,"economy"]=0 if df_res.iloc[l,10]==0 else df_res.iloc[l,9]/df_res.iloc[l,10]
        new.at[l,"rank"]=0 if df_res.iloc[l,7]==0 else df_res.iloc[l,6]/df_res.iloc[l,7]
        new.at[l,"kda"]=0 if df_res.iloc[l,12]==0 else df_res.iloc[l,11]/df_res.iloc[l,12]
        new.at[l,"hs"]=0 if df_res.iloc[l,14]==0 else df_res.iloc[l,13]/df_res.iloc[l,14]
    
    new.loc[:,'date'] = new.loc[:,'date'].astype(float)
    new.loc[:,'_map'] = new.loc[:,'_map'].astype(str)
    new.loc[:,'match_winner'] = new.loc[:,'match_winner'].astype(float)
    new.loc[:,'result_p1'] = new.loc[:,'result_p1'].astype(float)
    new.loc[:,'economy'] = new.loc[:,'economy'].astype(float)
    new.loc[:,'rank'] = new.loc[:,'rank'].astype(float)
    new.loc[:,'kda'] = new.loc[:,'kda'].astype(float)
    new.loc[:,'hs'] = new.loc[:,'hs'].astype(float)
    return new

In [2]:
def standardize_df(df):
    '''Standardize all the columns except the last one (target values).'''
    df_scaled = (df-df.mean())/df.std()
    df_scaled.iloc[:,0] = df.iloc[:,0]
    df_scaled.iloc[:,5] = df.iloc[:,5]
    return df_scaled

In [1]:
def standardize_zscore(df):
    '''Standardize all the columns except the last one (target values).'''
    df_scaled = (df-df.mean())/df.std()
    df_scaled.iloc[:,0] = df.iloc[:,0]
    df_scaled.iloc[:,5] = df.iloc[:,5]
    return df_scaled